In [1]:
from itertools import chain
from random import random

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

from society.action import Action
from society.agent import Agent, TrainableAgent
from society.simulations.weighted import WeightedNetworkSimulation
from society.strategies.gameplay.constant import AllC, AllD
from society.strategies.gameplay.qlearning import TabularQLearningGameplayStrategy
from society.strategies.gameplay.random import RandomGameplayStrategy
from society.strategies.gameplay.tft import TitForTat
from society.visualisation.network import *

plt.rcParams["figure.figsize"] = (10, 6)

In [2]:
def generate_population(communities, size):
    population = communities * size

    agents = [
        Agent(TabularQLearningGameplayStrategy(lookback=3, epsilon=0.1), i, population)
        for i in range(population)
    ]

    G = nx.connected_caveman_graph(communities, size)

    weights_matrix = np.zeros((population, population))
    for u, v, d in G.edges(data=True):
        weights_matrix[u, v] = weights_matrix[v, u] = 1.0
        try:
            d["weight"] = weights_matrix[u, v]
        except:
            d["weight"] = 0

    return agents, weights_matrix, G

In [3]:
def compute_policies(agents):
    policies = [
        agent.gameplay_strategy._q_table.argmax(axis=-1)
        for agent in agents
    ]

    return [
        "".join(
            [
                ("C", "D")[policy[i, j, k]]
                for i in range(4)
                for j in range(4)
                for k in range(4)
            ]
        )
        for policy in policies
    ]

def calculate_cooperativeness(history):
    count = history.count(Action.COOPERATE)

    return count / len(history)

In [4]:
# PAYOFF_LABELS = ["(C, C)", "(C, D)", "(D, C)", "(D, D)"]

PAYOFF_LABELS = ["R", "S", "T", "P"]

POPULATION = (2, 32)
ROUNDS = 25_000

run_rewards = []
run_cumulative_rewards = []
run_action_histories = []
run_reward_histories = []
run_policies = []

for run in range(20):
    # Generate a new population
    agents, weights_matrix, G = generate_population(*POPULATION)

    # Run a number of rounds
    sim = WeightedNetworkSimulation(agents, weights_matrix)

    # Run the simulation
    sim.reset()
    for i in tqdm(range(ROUNDS), desc=f"Run {run + 1}"):
        sim.play_round(train=True)

    # Store results
    run_rewards.append(sim.rewards)
    run_action_histories.append(sim.action_histories)
    run_reward_histories.append(sim.reward_histories)
    run_policies.append(compute_policies(agents))

    # Output statistics for the best and worst agents
    cumulative_rewards = [(i, sum(history)) for i, history in enumerate(sim.reward_histories)]
    cumulative_rewards.sort(key=lambda x: x[1], reverse=True)

    tqdm.write(
        f"BEST AGENT: {cumulative_rewards[0][1]} ({calculate_cooperativeness(list(chain(*sim.action_histories[cumulative_rewards[0][0]])))}, {run_policies[-1][cumulative_rewards[0][0]]})"
    )
    tqdm.write(
        f"WORST AGENT: {cumulative_rewards[-1][1]} ({calculate_cooperativeness(list(chain(*sim.action_histories[cumulative_rewards[-1][0]])))}, {run_policies[-1][cumulative_rewards[-1][0]]})"
    )


Run 1:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 171071 (0.6410590451757726, DCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCDCDCCCCCCCCCCCCCDCCCCCCCCCCCDCCC)
WORST AGENT: 134574 (0.9367217384227232, CCCCCDCCCCCCCDCCCCCDCCCCCCCCCDCCCCCCCCCCCCCCCDCCCCCDCCCCCCCCCCCC)


Run 2:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 174374 (0.5962838854666211, DCCCDCCCDCCCCCCCCCCCCCCCCCCCCCCCCCDCDCCCDCCCCCCCDCCCDCCCCCCCCCCC)
WORST AGENT: 132676 (0.9383452827114225, DCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCC)


Run 3:   0%|          | 0/25000 [00:00<?, ?it/s]

...

BEST AGENT: 171574 (0.5933346880715302, DCCCCCCCDCCCCCCCDCCCCCCCCCCCCCCCCCCCDCCDCCCCCCCCCCCCCCCCCCCCCCCC)
WORST AGENT: 132343 (0.9384880828183934, DCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCC)


Run 19:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 169560 (0.6091091091091091, DDCDCCCCDCCCCCCCDCCDCCCCCCDCCCDCCDCCCCCCCCCCCCCCCCCCCCCCDCCCCCCC)
WORST AGENT: 135330 (0.9392415884711678, CCCCCCDCCCCCCCCDCCDCCCCCCCCDCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCC)


Run 20:   0%|          | 0/25000 [00:00<?, ?it/s]

BEST AGENT: 160097 (0.7432594367884962, CCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCDCCCCCCCCCCCDCCC)
WORST AGENT: 140282 (0.9438964744242918, CCCCCCCCCCCCCDCCCCCCCCCCCDDCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCC)


In [5]:
from datetime import datetime
import pickle

with open(f"results - {datetime.isoformat(datetime.now()).replace(':', '-')}.pickle", "wb") as f:
    pickle.dump({
        "rewards": run_rewards,
        "cumulative_rewards": run_cumulative_rewards,
        # "action_histories": run_action_histories,
        "reward_histories": run_reward_histories,
        "policies": run_policies,
    }, f)


In [6]:
cumulative_reward_policy_ranks = {}
cumulative_reward_policies = {}

for policies, reward_histories in zip(run_policies, run_reward_histories):
    cumulative_rewards = [(i, sum(history)) for i, history in enumerate(reward_histories)]
    cumulative_rewards.sort(key=lambda x: x[1], reverse=True)

    for rank, (i, cumulative_reward) in enumerate(cumulative_rewards):
        policy = policies[i]

        if policy not in cumulative_reward_policy_ranks:
            cumulative_reward_policy_ranks[policy] = []
            cumulative_reward_policies[policy] = []

        cumulative_reward_policy_ranks[policy].append(rank + 1)
        cumulative_reward_policies[policy].append(cumulative_reward)

mean_cumulative_reward_policy_ranks = {policy: np.mean(ranks) for policy, ranks in cumulative_reward_policy_ranks.items()}
mean_cumulative_reward_policies = {policy: np.mean(rewards) for policy, rewards in cumulative_reward_policies.items()}

In [7]:
for policy in sorted(mean_cumulative_reward_policy_ranks, key=lambda x: mean_cumulative_reward_policy_ranks[x]):
    print(f"{mean_cumulative_reward_policy_ranks[policy]:<24} {policy}")

1.0                      DCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCDCDCCCCCCCCCCCCCDCCCCCCCCCCCDCCC
1.0                      DCCCDCCCDCCCCCCCCCCCCCCCCCCCCCCCCCDCDCCCDCCCCCCCDCCCDCCCCCCCCCCC
1.0                      DCDCCCCCCCDCCCCCDCCCCCCCCCCCCCCCCCCCDCCCCCDCCCCDDCCCCCCCCCCCCCCC
1.0                      DCCCDCDCDCCCCCCCDCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
1.0                      DDCCCCCCCCCCDCCCDCCCCCCCCCCCCCCCDCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCC
1.0                      DDCCCCDCDCCCDDCCDDCCDCCCCCCCCCCCDCCCDCCCCCCCCDCCCCCCCCCCCDCCCDCC
1.0                      DDCCCCCCDCCCCCCCDCCCCCCCDCCCCCCCDDCCCCCCDCCCCCCCCCCCCCCCCCCCCCCC
1.0                      DCDCCCCCCCCCCCCCDCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCC
1.0                      DCDDCCCCCCDCCCCCDCCDCCCCCCCCCCDCDCCCCCCDCCDCCCCCCCCCCCCCCCCCDCCC
1.0                      DCCCDCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCDCDCCCCCCCCCCCCC
1.0                      DCCCCCCCDCCCCCCCDCCCCCCCDCCCCCCCCDCCCCDCDCCCCCCDDCCCCCCCCCCCCCCC
1.0       

In [8]:
for policy in sorted(mean_cumulative_reward_policies, key=lambda x: mean_cumulative_reward_policies[x], reverse=True):
    print(f"{mean_cumulative_reward_policies[policy]:<24} {policy}")

191051.0                 DDDCCCCCCCDCCCCCDDCCCCCCCCCCCDCCDCCCCCCCCCDCCCCCDCCCCCCCCDCCCCCC
176237.0                 DCDCCCCCCCDCCCCCDCCCCCCCCCCCCCCCCCCCDCCCCCDCCCCDDCCCCCCCCCCCCCCC
175064.0                 DDCCCCCCDCCCCCCCDCCCCCCCDCCCCCCCDDCCCCCCDCCCCCCCCCCCCCCCCCCCCCCC
174374.0                 DCCCDCCCDCCCCCCCCCCCCCCCCCCCCCCCCCDCDCCCDCCCCCCCDCCCDCCCCCCCCCCC
174298.0                 DCCCDCDCDCCCCCCCDCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
174197.0                 DCCCDDCCDCCCCCCCCCCCCCCCCCCCCCCCDCCDCCCCCCCCCDCCDCCDCCCCCCCCCCCC
173745.0                 DDCCCCCCDCCCCCCCCCCCCCCCDCCCCCCCCCCCDCCCDCCCCCCCDCCCCCDDCCCCCDCC
173519.0                 DCCCDCCCDCCDDDCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
172580.0                 DDCCCCDCDCCCDDCCDDCCDCCCCCCCCCCCDCCCDCCCCCCCCDCCCCCCCCCCCDCCCDCC
172565.0                 DCCCCCCCDCCCCCCDDCCCCCCCCCCCCCCCDCCCCCCDCCCCCCCCCCCDCCCCCCCCCCCC
172518.0                 DCCCCCCCDDCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCDCCCCDCCCCCCCCCCCCDCCCCC
172186.0  